# Alternative Credit Scoring System

This notebook implements a comprehensive alternative credit scoring system that combines traditional credit data with gig economy metrics and behavioral factors to produce a credit score out of 1000.

## Key Components:
1. **Unified Feature Set**: Incorporates features from traditional and alternative data sources
2. **Weighted Scoring System**: Uses a detailed weighting system across four main categories
3. **ML Models**: Implements Logistic Regression and XGBoost with GPU acceleration
4. **Interactive Calculator**: Provides a user-friendly interface for testing profiles

## 1. Setup and Configuration

In [ ]:
# Install required packages
!pip install numpy pandas scikit-learn matplotlib seaborn xgboost ipywidgets -q

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, f1_score
import pickle
import os
import json
from ipywidgets import interact, widgets, Layout
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Check for XGBoost GPU availability
XGB_GPU_AVAILABLE = False
try:
    # Create a small test dataset
    X_test_gpu = np.random.random((10, 5))
    y_test_gpu = np.random.randint(0, 2, 10)
    xgb_test = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)
    xgb_test.fit(X_test_gpu, y_test_gpu)
    print('XGBoost GPU support available')
    XGB_GPU_AVAILABLE = True
except Exception as e:
    print(f'XGBoost GPU support not available: {e}')
    XGB_GPU_AVAILABLE = False

## 2. Data Generation

We'll generate sample data for our alternative credit scoring model.

In [ ]:
def generate_sample_data(num_records=2000):
    data = {}
    data['loan_amount'] = np.random.randint(1000, 50000, num_records)
    data['loan_term'] = np.random.choice([36, 60], num_records)
    data['interest_rate'] = np.random.uniform(5, 25, num_records)
    data['monthly_payment'] = data['loan_amount'] / data['loan_term'] * (1 + data['interest_rate']/100 / 12)**data['loan_term']
    data['credit_grade'] = np.random.choice(['A', 'B', 'C', 'D', 'E', 'F'], num_records, p=[0.2, 0.3, 0.25, 0.15, 0.05, 0.05])
    data['employment_length'] = np.random.randint(0, 11, num_records)
    data['annual_income'] = np.random.randint(20000, 200000, num_records)
    data['verification_status'] = np.random.choice(['Verified', 'Source Verified', 'Not Verified'], num_records, p=[0.4, 0.3, 0.3])
    data['home_ownership'] = np.random.choice(['MORTGAGE', 'RENT', 'OWN'], num_records, p=[0.45, 0.45, 0.1])
    data['delinq_2yrs'] = np.random.poisson(0.5, num_records)
    data['pub_rec'] = np.random.poisson(0.2, num_records)
    data['revol_util'] = np.random.uniform(0, 100, num_records)
    data['dti'] = np.random.uniform(5, 45, num_records)
    data['open_acc'] = np.random.randint(2, 30, num_records)
    data['total_acc'] = data['open_acc'] + np.random.randint(0, 20, num_records)
    data['inq_last_6mths'] = np.random.poisson(1, num_records)
    data['mort_acc'] = np.random.poisson(0.7, num_records)
    
    # Gig economy features
    data['gig_platforms_count'] = np.random.randint(0, 4, num_records)
    data['gig_platform_rating'] = np.random.uniform(3.0, 5.0, num_records)
    data['gig_platform_rating'] = np.where(data['gig_platforms_count'] == 0, 0, data['gig_platform_rating'])
    data['gig_completion_rate'] = np.random.uniform(0.7, 1.0, num_records)
    data['gig_completion_rate'] = np.where(data['gig_platforms_count'] == 0, 0, data['gig_completion_rate'])
    
    # Payment consistency features
    data['utility_payments_ontime'] = np.random.uniform(0.5, 1.0, num_records)
    data['rent_payments_ontime'] = np.random.uniform(0.5, 1.0, num_records)
    data['subscription_payments_ontime'] = np.random.uniform(0.5, 1.0, num_records)
    data['months_payment_history'] = np.random.randint(6, 120, num_records)
    data['late_payments_90d'] = np.random.poisson(0.3, num_records)
    
    # Asset value features
    data['bank_balance_avg'] = np.random.randint(500, 50000, num_records)
    data['bank_balance_min'] = data['bank_balance_avg'] * np.random.uniform(0.1, 0.8, num_records)
    data['investment_assets'] = np.random.randint(0, 100000, num_records)
    
    # Behavioral features
    data['cashflow_ratio'] = np.random.uniform(0.8, 2.5, num_records)
    data['savings_rate'] = np.random.uniform(0, 0.3, num_records)
    data['digital_footprint_score'] = np.random.randint(300, 850, num_records)
    data['shopping_categories'] = np.random.randint(1, 15, num_records)
    data['gambling_expenses'] = np.random.gamma(1, 50, num_records)
    data['education_level'] = np.random.randint(0, 5, num_records)
    
    # Target variable: 1 for default/rejected, 0 for approved/paid
    default_prob = {'A': 0.02, 'B': 0.05, 'C': 0.10, 'D': 0.20, 'E': 0.35, 'F': 0.5}
    data['is_default'] = [np.random.choice([0,1], p=[1-default_prob[grade], default_prob[grade]]) for grade in data['credit_grade']]
    
    df = pd.DataFrame(data)
    
    # Add some missing values randomly to simulate real-world data
    for col in df.columns:
        if df[col].dtype in [np.float64, np.int64] and col != 'is_default':
            mask = np.random.rand(len(df)) < 0.05 # 5% missing values
            df.loc[mask, col] = np.nan
    return df

# Generate sample data
df = generate_sample_data(2000)
print(f'Generated {len(df)} sample records.')

# Display basic info
print('\nDataset Info:')
df.info()
print('\nSample Data:')
display(df.head())

## 3. Alternative Credit Scorer Class

This class encapsulates the logic for feature engineering, scoring, and interpretation.

In [ ]:
class AlternativeCreditScorer:
    def __init__(self):
        self.category_weights = {
            'income_stability': 0.35,
            'payment_consistency': 0.30,
            'asset_value': 0.20,
            'behavioral_factors': 0.15
        }
        self.feature_weights = {
            'income_stability': {
                'employment_length': 0.20,
                'annual_income': 0.25,
                'income_to_loan_ratio': 0.15,
                'verification_status': 0.10,
                'gig_platforms_count': 0.10,
                'gig_platform_rating': 0.10,
                'gig_completion_rate': 0.10
            },
            'payment_consistency': {
                'utility_payments_ontime': 0.15,
                'rent_payments_ontime': 0.15,
                'subscription_payments_ontime': 0.10,
                'months_payment_history': 0.15,
                'late_payments_90d': 0.15,
                'delinq_2yrs': 0.15,
                'pub_rec': 0.10,
                'revol_util': 0.05
            },
            'asset_value': {
                'home_ownership': 0.25,
                'bank_balance_avg': 0.25,
                'bank_balance_min': 0.15,
                'investment_assets': 0.20,
                'mort_acc': 0.15
            },
            'behavioral_factors': {
                'dti': 0.20,
                'cashflow_ratio': 0.15,
                'savings_rate': 0.15,
                'digital_footprint_score': 0.10,
                'shopping_categories': 0.10,
                'gambling_expenses': 0.15,
                'education_level': 0.10,
                'open_acc': 0.05
            }
        }
        self.scaler = MinMaxScaler()

    def _normalize_feature(self, value, min_val, max_val, higher_is_better=True):
        value = np.clip(value, min_val, max_val)
        normalized = (value - min_val) / (max_val - min_val) if (max_val - min_val) != 0 else 0
        return normalized if higher_is_better else 1 - normalized

    def _map_categorical(self, value, mapping, default_score=0.0):
        return mapping.get(str(value).upper(), default_score)

    def engineer_features(self, profile):
        # Ensure profile is a Series for consistent access
        if isinstance(profile, dict):
            profile = pd.Series(profile)
            
        # Fill NaNs with reasonable defaults
        default_values = {
            'employment_length': 0, 'annual_income': 30000, 'loan_amount': 10000, 'verification_status': 'NOT VERIFIED',
            'gig_platforms_count': 0, 'gig_platform_rating': 0, 'gig_completion_rate': 0,
            'utility_payments_ontime': 0.5, 'rent_payments_ontime': 0.5, 'subscription_payments_ontime': 0.5,
            'months_payment_history': 0, 'late_payments_90d': 0, 'delinq_2yrs': 0, 'pub_rec': 0, 'revol_util': 50,
            'home_ownership': 'RENT', 'bank_balance_avg': 1000, 'bank_balance_min': 100, 'investment_assets': 0, 'mort_acc': 0,
            'dti': 30, 'cashflow_ratio': 1.0, 'savings_rate': 0.0, 'digital_footprint_score': 500,
            'shopping_categories': 5, 'gambling_expenses': 0, 'education_level': 1, 'open_acc': 5
        }
        profile = profile.fillna(value=default_values)
        
        # Handle missing keys
        for key, default_val in default_values.items():
            if key not in profile:
                profile[key] = default_val
                
        eng_features = {}

        # Income Stability
        eng_features['employment_length_norm'] = self._normalize_feature(profile['employment_length'], 0, 10)
        eng_features['annual_income_norm'] = self._normalize_feature(profile['annual_income'], 10000, 200000)
        income_to_loan = profile['annual_income'] / profile['loan_amount'] if profile['loan_amount'] > 0 else 0
        eng_features['income_to_loan_ratio_norm'] = self._normalize_feature(income_to_loan, 0.5, 10)
        verification_map = {'VERIFIED': 1.0, 'SOURCE VERIFIED': 0.7, 'NOT VERIFIED': 0.3}
        eng_features['verification_status_norm'] = self._map_categorical(profile['verification_status'], verification_map, 0.3)
        eng_features['gig_platforms_count_norm'] = self._normalize_feature(profile['gig_platforms_count'], 0, 5)
        eng_features['gig_platform_rating_norm'] = self._normalize_feature(profile['gig_platform_rating'], 0, 5)
        eng_features['gig_completion_rate_norm'] = self._normalize_feature(profile['gig_completion_rate'], 0, 1)

        # Payment Consistency
        eng_features['utility_payments_ontime_norm'] = self._normalize_feature(profile['utility_payments_ontime'], 0, 1)
        eng_features['rent_payments_ontime_norm'] = self._normalize_feature(profile['rent_payments_ontime'], 0, 1)
        eng_features['subscription_payments_ontime_norm'] = self._normalize_feature(profile['subscription_payments_ontime'], 0, 1)
        eng_features['months_payment_history_norm'] = self._normalize_feature(profile['months_payment_history'], 0, 120)
        eng_features['late_payments_90d_norm'] = self._normalize_feature(profile['late_payments_90d'], 0, 5, higher_is_better=False)
        eng_features['delinq_2yrs_norm'] = self._normalize_feature(profile['delinq_2yrs'], 0, 5, higher_is_better=False)
        eng_features['pub_rec_norm'] = self._normalize_feature(profile['pub_rec'], 0, 3, higher_is_better=False)
        eng_features['revol_util_norm'] = self._normalize_feature(profile['revol_util'], 0, 100, higher_is_better=False)

        # Asset Value
        home_ownership_map = {'OWN': 1.0, 'MORTGAGE': 0.7, 'RENT': 0.3, 'OTHER': 0.2}
        eng_features['home_ownership_norm'] = self._map_categorical(profile['home_ownership'], home_ownership_map, 0.2)
        eng_features['bank_balance_avg_norm'] = self._normalize_feature(profile['bank_balance_avg'], 0, 50000)
        eng_features['bank_balance_min_norm'] = self._normalize_feature(profile['bank_balance_min'], 0, 10000)
        eng_features['investment_assets_norm'] = self._normalize_feature(profile['investment_assets'], 0, 100000)
        eng_features['mort_acc_norm'] = self._normalize_feature(profile['mort_acc'], 0, 4)

        # Behavioral Factors
        eng_features['dti_norm'] = self._normalize_feature(profile['dti'], 0, 50, higher_is_better=False)
        eng_features['cashflow_ratio_norm'] = self._normalize_feature(profile['cashflow_ratio'], 0.5, 3)
        eng_features['savings_rate_norm'] = self._normalize_feature(profile['savings_rate'], 0, 0.5)
        eng_features['digital_footprint_score_norm'] = self._normalize_feature(profile['digital_footprint_score'], 300, 850)
        eng_features['shopping_categories_norm'] = self._normalize_feature(profile['shopping_categories'], 1, 20)
        eng_features['gambling_expenses_norm'] = self._normalize_feature(profile['gambling_expenses'], 0, 1000, higher_is_better=False)
        eng_features['education_level_norm'] = self._normalize_feature(profile['education_level'], 0, 4)
        eng_features['open_acc_norm'] = self._normalize_feature(profile['open_acc'], 0, 40)

        return eng_features

    def calculate_category_scores(self, engineered_features):
        category_scores = {}
        for category, f_weights in self.feature_weights.items():
            score = 0
            for feature_name_base, weight in f_weights.items():
                norm_feature_name = f'{feature_name_base}_norm'
                feature_value = engineered_features.get(norm_feature_name, 0)
                score += feature_value * weight
            category_scores[category] = score
        return category_scores

    def calculate_final_score(self, category_scores):
        final_score = 0
        for category, weight in self.category_weights.items():
            final_score += category_scores.get(category, 0) * weight
        return int(final_score * 1000)

    def get_score_interpretation(self, final_score):
        if final_score >= 900: grade, desc, rec, rates = 'A+', 'Excellent', 'Approved', '4-7%'
        elif final_score >= 800: grade, desc, rec, rates = 'A', 'Very Good', 'Approved', '5-8%'
        elif final_score >= 700: grade, desc, rec, rates = 'B+', 'Good', 'Approved', '6-9%'
        elif final_score >= 650: grade, desc, rec, rates = 'B', 'Fair Good', 'Approved', '7-11%'
        elif final_score >= 600: grade, desc, rec, rates = 'C+', 'Fair', 'Conditionally Approved', '9-14%'
        elif final_score >= 550: grade, desc, rec, rates = 'C', 'Poor Fair', 'Conditionally Approved with Restrictions', '11-16%'
        elif final_score >= 500: grade, desc, rec, rates = 'D+', 'Poor', 'Conditionally Approved with High Restrictions', '14-18%'
        else: grade, desc, rec, rates = 'D', 'Very Poor', 'Denied', 'N/A'
        return {'grade': grade, 'description': desc, 'recommendation': rec, 'rate_range': rates}

    def score_profile(self, profile):
        engineered = self.engineer_features(profile)
        category_scores_raw = self.calculate_category_scores(engineered)
        final_score = self.calculate_final_score(category_scores_raw)
        interpretation = self.get_score_interpretation(final_score)
        
        # Category scores as percentage for display
        category_scores_display = {cat: int(score * 100) for cat, score in category_scores_raw.items()}
        
        return {
            'final_score': final_score,
            'interpretation': interpretation,
            'category_scores': category_scores_display,
            'engineered_features': engineered
        }

# Initialize scorer
scorer = AlternativeCreditScorer()

## 4. Test Scorer with Sample Profiles

In [ ]:
# Define sample profiles
sample_good_profile = {
    'loan_amount': 10000, 'loan_term': 36, 'interest_rate': 7.5, 'monthly_payment': 311,
    'employment_length': 8, 'annual_income': 120000, 'verification_status': 'Verified',
    'gig_platforms_count': 2, 'gig_platform_rating': 4.8, 'gig_completion_rate': 0.95,
    'utility_payments_ontime': 0.98, 'rent_payments_ontime': 1.0, 'subscription_payments_ontime': 1.0,
    'months_payment_history': 60, 'late_payments_90d': 0, 'delinq_2yrs': 0, 'pub_rec': 0, 'revol_util': 20,
    'home_ownership': 'MORTGAGE', 'bank_balance_avg': 25000, 'bank_balance_min': 10000, 'investment_assets': 50000, 'mort_acc': 1,
    'dti': 15, 'cashflow_ratio': 1.8, 'savings_rate': 0.20, 'digital_footprint_score': 800,
    'shopping_categories': 10, 'gambling_expenses': 10, 'education_level': 3, 'open_acc': 12
}

sample_average_profile = {
    'loan_amount': 20000, 'loan_term': 60, 'interest_rate': 12.0, 'monthly_payment': 445,
    'employment_length': 3, 'annual_income': 60000, 'verification_status': 'Source Verified',
    'gig_platforms_count': 1, 'gig_platform_rating': 4.2, 'gig_completion_rate': 0.88,
    'utility_payments_ontime': 0.90, 'rent_payments_ontime': 0.92, 'subscription_payments_ontime': 0.95,
    'months_payment_history': 30, 'late_payments_90d': 1, 'delinq_2yrs': 1, 'pub_rec': 0, 'revol_util': 55,
    'home_ownership': 'RENT', 'bank_balance_avg': 5000, 'bank_balance_min': 1000, 'investment_assets': 10000, 'mort_acc': 0,
    'dti': 28, 'cashflow_ratio': 1.2, 'savings_rate': 0.08, 'digital_footprint_score': 650,
    'shopping_categories': 7, 'gambling_expenses': 50, 'education_level': 2, 'open_acc': 8
}

sample_poor_profile = {
    'loan_amount': 15000, 'loan_term': 36, 'interest_rate': 18.5, 'monthly_payment': 546,
    'employment_length': 1, 'annual_income': 35000, 'verification_status': 'Not Verified',
    'gig_platforms_count': 0, 'gig_platform_rating': 0, 'gig_completion_rate': 0,
    'utility_payments_ontime': 0.70, 'rent_payments_ontime': 0.75, 'subscription_payments_ontime': 0.80,
    'months_payment_history': 12, 'late_payments_90d': 3, 'delinq_2yrs': 2, 'pub_rec': 1, 'revol_util': 85,
    'home_ownership': 'RENT', 'bank_balance_avg': 800, 'bank_balance_min': 50, 'investment_assets': 500, 'mort_acc': 0,
    'dti': 42, 'cashflow_ratio': 0.9, 'savings_rate': 0.01, 'digital_footprint_score': 450,
    'shopping_categories': 3, 'gambling_expenses': 200, 'education_level': 1, 'open_acc': 6
}

# Score the profiles
print("--- Good Profile Score ---")
good_score_details = scorer.score_profile(sample_good_profile)
print(f"Score: {good_score_details['final_score']}")
print(f"Grade: {good_score_details['interpretation']['grade']} - {good_score_details['interpretation']['description']}")
print(f"Recommendation: {good_score_details['interpretation']['recommendation']}")
print(f"Rate Range: {good_score_details['interpretation']['rate_range']}")
print("Category Scores:")
for category, score in good_score_details['category_scores'].items():
    print(f"  {category}: {score}%")

print("\n--- Average Profile Score ---")
avg_score_details = scorer.score_profile(sample_average_profile)
print(f"Score: {avg_score_details['final_score']}")
print(f"Grade: {avg_score_details['interpretation']['grade']} - {avg_score_details['interpretation']['description']}")
print(f"Recommendation: {avg_score_details['interpretation']['recommendation']}")
print(f"Rate Range: {avg_score_details['interpretation']['rate_range']}")
print("Category Scores:")
for category, score in avg_score_details['category_scores'].items():
    print(f"  {category}: {score}%")

print("\n--- Poor Profile Score ---")
poor_score_details = scorer.score_profile(sample_poor_profile)
print(f"Score: {poor_score_details['final_score']}")
print(f"Grade: {poor_score_details['interpretation']['grade']} - {poor_score_details['interpretation']['description']}")
print(f"Recommendation: {poor_score_details['interpretation']['recommendation']}")
print(f"Rate Range: {poor_score_details['interpretation']['rate_range']}")
print("Category Scores:")
for category, score in poor_score_details['category_scores'].items():
    print(f"  {category}: {score}%")

## 5. Prepare Data for Machine Learning Models

In [ ]:
# Apply feature engineering to the entire dataset
engineered_df_list = []
for index, row in df.iterrows():
    profile_score = scorer.score_profile(row)
    engineered_df_list.append(profile_score['engineered_features'])

X_engineered = pd.DataFrame(engineered_df_list)
y = df['is_default']

# Ensure X_engineered and y have the same index for splitting
X_engineered.index = y.index

print(f'Shape of engineered features (X): {X_engineered.shape}')
print(f'Shape of target variable (y): {y.shape}')
print('\nEngineered Features Head:')
display(X_engineered.head())

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_engineered, y, test_size=0.2, random_state=42, stratify=y)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

# Scale numerical features
scaler_ml = StandardScaler()
X_train_scaled = scaler_ml.fit_transform(X_train)
X_test_scaled = scaler_ml.transform(X_test)

## 6. Train and Evaluate Machine Learning Models

In [ ]:
# Logistic Regression
print("--- Logistic Regression ---")
log_reg = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')
log_reg.fit(X_train_scaled, y_train)
y_pred_log_reg = log_reg.predict(X_test_scaled)
y_proba_log_reg = log_reg.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred_log_reg))
print(f'ROC AUC: {roc_auc_score(y_test, y_proba_log_reg):.4f}')
print(f'Accuracy: {accuracy_score(y_test, y_pred_log_reg):.4f}')
print(f'F1 Score: {f1_score(y_test, y_pred_log_reg):.4f}')

In [ ]:
# XGBoost Classifier
print("\n--- XGBoost Classifier ---")
xgb_clf = xgb.XGBClassifier(
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss', 
    tree_method='hist' if not XGB_GPU_AVAILABLE else 'gpu_hist',
    gpu_id=0 if XGB_GPU_AVAILABLE else -1,
    scale_pos_weight= (len(y_train) - sum(y_train)) / sum(y_train)
)
xgb_clf.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_clf.predict(X_test_scaled)
y_proba_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred_xgb))
print(f'ROC AUC: {roc_auc_score(y_test, y_proba_xgb):.4f}')
print(f'Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}')
print(f'F1 Score: {f1_score(y_test, y_pred_xgb):.4f}')

## 7. Save Trained Models

In [ ]:
# Create a directory to save models if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save Logistic Regression model and scaler
with open('models/logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(log_reg, f)
with open('models/ml_scaler.pkl', 'wb') as f:
    pickle.dump(scaler_ml, f)
print('Logistic Regression model and ML scaler saved.')

# Save XGBoost model
with open('models/xgboost_model.pkl', 'wb') as f:
    pickle.dump(xgb_clf, f)
print('XGBoost model saved.')

# Save the AlternativeCreditScorer instance
with open('models/alternative_credit_scorer.pkl', 'wb') as f:
    pickle.dump(scorer, f)
print('AlternativeCreditScorer instance saved.')

## 8. Interactive Credit Score Calculator

In [ ]:
# Define widgets for input features
loan_amount = widgets.IntSlider(value=15000, min=1000, max=100000, step=500, description='Loan Amount ($):')
employment_length = widgets.IntSlider(value=5, min=0, max=40, step=1, description='Employment Length (Years):')
annual_income = widgets.IntSlider(value=60000, min=10000, max=500000, step=1000, description='Annual Income ($):')
verification_status = widgets.Dropdown(options=['Verified', 'Source Verified', 'Not Verified'], value='Source Verified', description='Income Verification:')
home_ownership = widgets.Dropdown(options=['MORTGAGE', 'RENT', 'OWN', 'OTHER'], value='RENT', description='Home Ownership:')
delinq_2yrs = widgets.IntSlider(value=0, min=0, max=10, step=1, description='Delinquencies (2 Yrs):')
pub_rec = widgets.IntSlider(value=0, min=0, max=5, step=1, description='Public Records:')
revol_util = widgets.FloatSlider(value=30.0, min=0.0, max=100.0, step=0.1, description='Revolving Utilization (%):')
dti = widgets.FloatSlider(value=25.0, min=0.0, max=100.0, step=0.1, description='Debt-to-Income Ratio (%):')
gig_platforms_count = widgets.IntSlider(value=1, min=0, max=5, step=1, description='Gig Platforms Count:')
gig_platform_rating = widgets.FloatSlider(value=4.0, min=0.0, max=5.0, step=0.1, description='Avg Gig Platform Rating:')
gig_completion_rate = widgets.FloatSlider(value=0.85, min=0.0, max=1.0, step=0.01, description='Gig Completion Rate:')

# Output widget for results
output_area = widgets.Output()

def calculate_score(loan_amount, employment_length, annual_income, verification_status, home_ownership, 
                    delinq_2yrs, pub_rec, revol_util, dti, gig_platforms_count, gig_platform_rating, gig_completion_rate):
    with output_area:
        clear_output(wait=True)
        profile = {
            'loan_amount': loan_amount,
            'employment_length': employment_length,
            'annual_income': annual_income,
            'verification_status': verification_status,
            'home_ownership': home_ownership,
            'delinq_2yrs': delinq_2yrs,
            'pub_rec': pub_rec,
            'revol_util': revol_util,
            'dti': dti,
            'gig_platforms_count': gig_platforms_count,
            'gig_platform_rating': gig_platform_rating,
            'gig_completion_rate': gig_completion_rate,
            # Default values for other features
            'utility_payments_ontime': 0.90,
            'rent_payments_ontime': 0.90,
            'subscription_payments_ontime': 0.90,
            'months_payment_history': 24,
            'late_payments_90d': 0,
            'bank_balance_avg': 5000,
            'bank_balance_min': 1000,
            'investment_assets': 10000,
            'mort_acc': 0,
            'cashflow_ratio': 1.2,
            'savings_rate': 0.10,
            'digital_footprint_score': 650,
            'shopping_categories': 7,
            'gambling_expenses': 50,
            'education_level': 2,
            'open_acc': 8
        }
        score_details = scorer.score_profile(profile)
        print("--- Credit Score Results ---")
        print(f"Final Score: {score_details['final_score']}")
        print(f"Grade: {score_details['interpretation']['grade']} ({score_details['interpretation']['description']})")
        print(f"Recommendation: {score_details['interpretation']['recommendation']}")
        print(f"Indicative Rate Range: {score_details['interpretation']['rate_range']}")
        print("\nCategory Scores:")
        for category, score_val in score_details['category_scores'].items():
            print(f"  {category.replace('_', ' ').title()}: {score_val} / 100")

# Create interactive widget
interactive_calculator = widgets.interactive(
    calculate_score,
    loan_amount=loan_amount,
    employment_length=employment_length,
    annual_income=annual_income,
    verification_status=verification_status,
    home_ownership=home_ownership,
    delinq_2yrs=delinq_2yrs,
    pub_rec=pub_rec,
    revol_util=revol_util,
    dti=dti,
    gig_platforms_count=gig_platforms_count,
    gig_platform_rating=gig_platform_rating,
    gig_completion_rate=gig_completion_rate
)

# Display the interactive calculator
print("Alternative Credit Score Calculator")
print("Adjust the sliders to see how different factors affect the credit score.")
display(interactive_calculator)
display(output_area)